In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
import collections
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import tensorboard

import nest_asyncio
nest_asyncio.apply()

2022-01-10 11:43:52.508032: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-10 11:43:52.508049: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
df = pd.read_csv(r"/home/bas/fed/df_combined.csv")

In [4]:
df.loc[(df.label == '-   Benign   -'), 'label'] = 'Benign'
df['label'].value_counts()
def drop_columns(df, columns):
    temp = df.copy()
    for column in columns:
        temp = temp[temp['label'] != column]
    return temp

columns_to_drop = [
                    "Attack",
                    "C&C-HeartBeat",
                    "C&C-FileDownload",
                    "C&C-Torii",
                    "FileDownload",
                    "C&C-HeartBeat-FileDownload",
                    "C&C-Mirai",
                    "PartOfAHorizontalPortScan"]

df = drop_columns(df, columns_to_drop)
df['label'].value_counts()

Okiru     262690
Benign    199756
DDoS      138777
C&C        15100
Name: label, dtype: int64

In [188]:
df

,Unnamed: 0,ts,id.orig_h,id.orig_p,id.resp_h,id.resp_p,duration,orig_bytes,resp_bytes,history,...,conn_state_SF,conn_state_SH,conn_state_SHR,service_-,service_dhcp,service_dns,service_http,service_irc,service_ssh,service_ssl
3,3,1.525880e+09,192.168.100.103,34243.0,147.7.65.203,49560.0,2.998804,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
4,4,1.525880e+09,192.168.100.103,34840.0,145.164.35.6,21288.0,0.000000,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
14,14,1.525880e+09,192.168.100.103,34243.0,147.7.65.203,49560.0,0.000000,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
27,27,1.525880e+09,192.168.100.103,34243.0,147.7.65.203,49560.0,0.000000,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
44,44,1.525880e+09,192.168.100.103,33685.0,197.224.138.9,34797.0,2.998560,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446616,122,1.562165e+09,0.0.0.0,68.0,255.255.255.255,67.0,-1.000000,-1.0,-1.0,D,...,0,0,0,0,1,0,0,0,0,0
1446617,123,1.562165e+09,::,143.0,ff02::16,0.0,-1.000000,-1.0,-1.0,-,...,0,0,0,1,0,0,0,0,0,0
1446618,124,1.562165e+09,::,135.0,ff02::1:ffd5:cdf,136.0,-1.000000,-1.0,-1.0,-,...,0,0,0,1,0,0,0,0,0,0
1446619,125,1.562165e+09,fe80::5bcc:698e:39d5:cdf,143.0,ff02::16,0.0,-1.000000,-1.0,-1.0,-,...,0,0,0,1,0,0,0,0,0,0


In [5]:
#id.orig_h is hier nodig, wordt later gedropt voordat het training in gaat

X = df[['Unnamed: 0', 'ts','id.orig_p', 'id.orig_h',
       'duration', 'orig_bytes', 'resp_bytes', 'orig_pkts',
       'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp',
       'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ',
       'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR',
       'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2',
       'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR',
       'service_-', 'service_dhcp', 'service_dns', 'service_http',
       'service_irc', 'service_ssh', 'service_ssl']]

In [190]:
X.shape


(616323, 34)

In [237]:
dummies = pd.get_dummies(df['label']).values
dummies.shape

(616323, 4)

In [6]:
Y = df[['id.orig_h','label']]
Y.shape
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, random_state=42)
X_train

,Unnamed: 0,ts,id.orig_p,id.orig_h,duration,orig_bytes,resp_bytes,orig_pkts,orig_ip_bytes,resp_pkts,...,conn_state_SF,conn_state_SH,conn_state_SHR,service_-,service_dhcp,service_dns,service_http,service_irc,service_ssh,service_ssl
649003,8978,1.545404e+09,34114.0,192.168.1.196,0.000000,0.0,0.0,1.0,60.0,0.0,...,0,0,0,1,0,0,0,0,0,0
239587,39589,1.532102e+09,5526.0,192.168.100.108,0.000000,0.0,0.0,1.0,40.0,0.0,...,0,0,0,1,0,0,0,0,0,0
655790,15765,1.545404e+09,41580.0,192.168.1.196,0.000000,0.0,0.0,1.0,60.0,0.0,...,0,0,0,1,0,0,0,0,0,0
639185,22302,1.545483e+09,56256.0,192.168.1.195,3.138219,0.0,0.0,3.0,180.0,0.0,...,0,0,0,1,0,0,0,0,0,0
479890,69494,1.536227e+09,17832.0,192.168.100.111,-1.000000,-1.0,-1.0,2.0,80.0,0.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262273,62275,1.532102e+09,5526.0,192.168.100.108,0.000000,0.0,0.0,1.0,40.0,0.0,...,0,0,0,1,0,0,0,0,0,0
632710,15827,1.545467e+09,65279.0,192.168.1.195,0.000000,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
739378,99353,1.545404e+09,45586.0,192.168.1.196,0.000000,0.0,0.0,1.0,60.0,0.0,...,0,0,0,1,0,0,0,0,0,0
283964,83966,1.532102e+09,5526.0,192.168.100.108,0.000000,0.0,0.0,1.0,40.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [7]:
#Dit kan beter, maar werkt

Y_train.loc[(df.label == 'Benign'), 'label'] = 0
Y_train.loc[(df.label == 'Okiru'), 'label'] = 1
Y_train.loc[(df.label == 'DDoS'), 'label'] = 2
Y_train.loc[(df.label == 'C&C'), 'label'] = 3
Y_train['label'] = np.asarray(Y_train['label']).astype('float32')

In [16]:
bigips = X['id.orig_h'].value_counts()[:10]
bigips

192.168.1.198      143025
192.168.1.195      122891
192.168.1.196       99973
192.168.100.108     99802
192.168.100.111     50985
192.168.100.103     45588
192.168.1.197       32017
192.168.100.113     13665
192.168.2.5          2277
192.168.2.3           979
Name: id.orig_h, dtype: int64

In [110]:
df_node = df[df['id.orig_h']=='192.168.100.103']
df_node['label'].value_counts()


Benign    45588
Name: label, dtype: int64

In [8]:
def get_client_training_data(id):
    x_node = X_train[X_train['id.orig_h']==id]
    x_node = x_node.drop('id.orig_h', axis=1)
    y_node = Y_train[Y_train['id.orig_h']==id]
    y_node = y_node.drop('id.orig_h', axis=1)

    x_node_ds = tf.data.Dataset.from_tensors(collections.OrderedDict(x=x_node, y=y_node))
    return x_node_ds

In [9]:
client_ids = df['id.orig_h']
client_set = set(client_ids)

In [113]:
#train_data = tff.simulation.datasets.ClientData.from_clients_and_fn(client_set, get_client_data_train)
train_data = tff.simulation.datasets.ClientData.from_clients_and_fn(['192.168.1.196', '192.168.1.195', '192.168.1.198', '192.168.100.108', '192.168.100.111', '192.168.100.103'], get_client_training_data)
# train_data = tff.simulation.datasets.ClientData.from_clients_and_fn(['192.168.1.196', '192.168.1.195', '192.168.1.198'], get_client_data_train)

#train_data = tff.simulation.datasets.ClientData.from_clients_and_fn(['192.168.100.111'], get_client_data_train)



def make_federated_data(client_data, client_ids):

  for x in client_ids:
    data=client_data.create_tf_dataset_for_client(x)
    print(data)

  return [
      client_data.create_tf_dataset_for_client(x)
      for x in client_ids
  ]

sample_clients = train_data.client_ids[:6]

federated_train_data = make_federated_data(train_data, sample_clients)

# example_dataset=federated_train_data.create_tf_dataset_for_client('192.168.1.198')

example_dataset = train_data.create_tf_dataset_for_client(train_data.client_ids[0])
    

<TensorDataset shapes: OrderedDict([(x, (80174, 33)), (y, (80174, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.float32)])>
<TensorDataset shapes: OrderedDict([(x, (98149, 33)), (y, (98149, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.float32)])>
<TensorDataset shapes: OrderedDict([(x, (114312, 33)), (y, (114312, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.float32)])>
<TensorDataset shapes: OrderedDict([(x, (79930, 33)), (y, (79930, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.float32)])>
<TensorDataset shapes: OrderedDict([(x, (40775, 33)), (y, (40775, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.float32)])>
<TensorDataset shapes: OrderedDict([(x, (36423, 33)), (y, (36423, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.float32)])>


In [114]:
print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

Number of client datasets: 6
First dataset: <TensorDataset shapes: OrderedDict([(x, (80174, 33)), (y, (80174, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.float32)])>


In [115]:
def create_keras_model():
    return tf.keras.models.Sequential([
      tf.keras.layers.Dense(64, input_shape=(33,), activation='relu'),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(4, activation='softmax'),
  ])

model = create_keras_model()
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 64)                2176      
_________________________________________________________________
dense_20 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_21 (Dense)             (None, 4)                 260       
Total params: 6,596
Trainable params: 6,596
Non-trainable params: 0
_________________________________________________________________


In [116]:
input_spec = collections.OrderedDict(
    x=tf.TensorSpec(dtype=tf.float64, shape=[None, 33]),
    y=tf.TensorSpec(dtype=tf.float32, shape=[None, 1]),
)

def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [117]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.08),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [58]:
str(iterative_process.initialize.type_signature)

'( -> <model=<trainable=<float32[33,64],float32[64],float32[64,64],float32[64],float32[64,4],float32[4]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [118]:
state = iterative_process.initialize()

In [60]:
print(iterative_process.next.type_signature)

(<server_state=<model=<trainable=<float32[33,64],float32[64],float32[64,64],float32[64],float32[64,4],float32[4]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER,federated_dataset={<x=float64[?,33],y=float32[?,1]>*}@CLIENTS> -> <<model=<trainable=<float32[33,64],float32[64],float32[64,64],float32[64],float32[64,4],float32[4]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER,<broadcast=<>,aggregation=<mean_value=<>,mean_weight=<>>,train=<sparse_categorical_accuracy=float32,loss=float32>,stat=<num_examples=int64>>@SERVER>)


In [61]:
state, metrics = iterative_process.next(state, federated_train_data)
print('round  1, metrics={}'.format(metrics))

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.31879678), ('loss', 10.97972)])), ('stat', OrderedDict([('num_examples', 449763)]))])


In [119]:
NUM_ROUNDS = 15
for round_num in range(2, NUM_ROUNDS):
  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.2462697), ('loss', 12.148722)])), ('stat', OrderedDict([('num_examples', 449763)]))])
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.42282936), ('loss', 9.302907)])), ('stat', OrderedDict([('num_examples', 449763)]))])
round  4, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.31879678), ('loss', 10.979721)])), ('stat', OrderedDict([('num_examples', 449763)]))])
round  5, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.42282936), ('loss', 9

In [120]:
logdir = "/tmp/logs/scalars/training/"
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()

In [121]:
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    for name, value in metrics['train'].items():
      tf.summary.scalar(name, value, step=round_num)

In [125]:
!ls {logdir}
%load_ext tensorboard
%tensorboard --logdir {logdir} --port=0

events.out.tfevents.1641808564.UbuntuBox.4685.216043.v2
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 33455 (pid 9182), started 0:01:05 ago. (Use '!kill 9182' to kill it.)